In [2]:
import numpy as np
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms
import os
import matplotlib.pyplot as plt
import ast

In [3]:
model = torch.hub.load('pytorch/vision:v0.8.2', 'alexnet', pretrained=True)
# model.eval()

Using cache found in C:\Users\mgina/.cache\torch\hub\pytorch_vision_v0.8.2


In [4]:
# Load data: if Windows
path='../Image/All_Cropped'

images={}

directory=['Mountain', 'Beach', 
           'Mug', 'Banana', 
           'Car', 'Plane', 
           'Lighthouse', 'Church']

for dir in directory:
    images[dir]=[file for file in os.listdir(path+'/'+dir) if file.endswith(('jpeg', 'jpg'))]

Compute mean and std of all images (cached)

In [220]:
# preprocess = transforms.Compose([
# #     transforms.Resize(256),
# #     transforms.CenterCrop(224),
#     transforms.ToTensor(),
# ])
# all_images = []
# for cate, filename in images.items():
#     for f in filename:
#         dir=os.path.join(path,cate,f)
#         img=preprocess(Image.open(dir))
# #         if img.shape[1] != 256 or img.shape[2] != 256:
# #             print(img.shape)
# #             plt.imshow(Image.open(dir))
# #             plt.show()
#         all_images.append(img)
# all_images = torch.stack(all_images).numpy()
# print(all_images.shape)
# mean = np.mean(all_images, axis=(0,2,3))
# std = np.std(all_images, axis=(0,2,3))

In [5]:
# preprocess = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std),
# ])
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # this should be the mean and std for alexnet training dataset
])

In [6]:
# read labels 
with open("../imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# read labels to wordnet synsets
ltw = pd.read_json('../imagenet_label_to_wordnet_synset.json').T

In [7]:
ltw.columns

Index(['id', 'label', 'uri'], dtype='object')

In [8]:
#type prob cate+filename
all_img={'type':[], 'dir':[],'id_labels':[],'readable_labels':[]}

In [9]:
for cate, filename in images.items():
    for f in filename:
        dir=os.path.join(path,cate,f)
        img=Image.open(dir)
        #print(dir)
        input_tensor=preprocess(img)
        input_batch = input_tensor.unsqueeze(0)

        if torch.cuda.is_available():
            input_batch= input_batch.to('cuda')
            model.to('cuda')

        with torch.no_grad():
            output = model(input_batch)
        
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        top10_prob, top10_catid = torch.topk(probabilities, 10)
#         typ_score=round(float(probabilities.max()*100),1)
        
        # Show top categories per image
        top10_prob, top10_catid = torch.topk(probabilities, 10)
        temp1 = {}
        temp2 = {}
        for i in range(top10_prob.size(0)):
            temp1[categories[top10_catid[i].item()]] = top10_prob[i].item()
            temp2[ltw['id'][top10_catid[i].item()]] = top10_prob[i].item()

        all_img['dir'].append(dir)
#         all_img['typicality_score'].append(typ_score)
        all_img['readable_labels'].append(temp1)
        all_img['id_labels'].append(temp2)
        if 'Mountain' in cate:
            all_img['type'].append('Mountain')
        if 'Beach' in cate:
            all_img['type'].append('Beach')
        if 'Mug' in cate:
            all_img['type'].append('Mug')
        if 'Banana' in cate:
            all_img['type'].append('Banana')
        if 'Car' in cate:
            all_img['type'].append('Car')
        if 'Plane' in cate:
            all_img['type'].append('Plane')
        if 'Lighthouse' in cate:
            all_img['type'].append('Lighthouse')
        if 'Church' in cate:
            all_img['type'].append('Church')

#         print('Typicality of ',f, ' = ',probabilities.max()*100)

In [10]:
all_img_df=pd.DataFrame(all_img)
all_img_df.head()

,type,dir,id_labels,readable_labels
0,Mountain,../Image/All_Cropped\Mountain\1.jpg,"{'09193705-n': 0.18738645315170288, '09468604-...","{'alp': 0.18738645315170288, 'valley': 0.06126..."
1,Mountain,../Image/All_Cropped\Mountain\10.jpg,"{'09468604-n': 0.6145076155662537, '09193705-n...","{'valley': 0.6145076155662537, 'alp': 0.135453..."
2,Mountain,../Image/All_Cropped\Mountain\11.jpg,"{'09472597-n': 0.7545696496963501, '09193705-n...","{'volcano': 0.7545696496963501, 'alp': 0.18182..."
3,Mountain,../Image/All_Cropped\Mountain\12.jpg,"{'09193705-n': 0.5299694538116455, '09468604-n...","{'alp': 0.5299694538116455, 'valley': 0.397528..."
4,Mountain,../Image/All_Cropped\Mountain\13.jpg,"{'09246464-n': 0.6374799013137817, '09399592-n...","{'cliff': 0.6374799013137817, 'promontory': 0...."


In [82]:
# all_img_df['label'] = all_img_df['top5_labels'].apply(lambda x: list(x.keys())[0])

In [83]:
# all_img_df['7_score']=round(all_img_df['typicality_score']*7/100,1)
# all_img_df.head()

In [84]:
# score=[]
# for i in range(5):
#     dir=all_img_df['dir'][i]
#     score.append([all_img_df['typicality_score'][i], all_img_df['7_score'][i]])
#     img=Image.open(dir)
#     plt.imshow(img)
#     plt.title(score[i])
#     plt.show()

In [85]:
# all_img_df.to_csv('alexnet_scores_wordnet_id.csv')